In [ ]:
#!pip install langchain==0.0.228
#!pip install cohere
# !pip install python-dotenv
#!pip install unstructured
#!pip install chromadb==0.3.26
#!pip install streamlit
#!pip install streamlit_chat


In [1]:
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import Cohere
from langchain.chains import RetrievalQA , LLMChain
from langchain.document_loaders import DirectoryLoader
from langchain import PromptTemplate,FewShotPromptTemplate
import os
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector



In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY', 'YourAPIKeyIfNotSet')

In [3]:
cohere_api_key = os.getenv('COHERE_API_KEY', 'YourAPIKeyIfNotSet')

In [33]:
inh_loader = DirectoryLoader('./inheritance', glob='*.txt')
div_loader = DirectoryLoader('./divorce', glob='*.txt')



In [34]:
inh_documents = inh_loader.load()
div_documents = div_loader.load()

In [ ]:
print(len(inh_documents))
print(len(div_documents))

In [35]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n","\n"],chunk_size=100, chunk_overlap=0)

In [36]:
inh_chunks = text_splitter.split_documents(inh_documents)
div_chunks = text_splitter.split_documents(div_documents)



In [38]:
inh_chunks[10:20]

[Document(page_content='\nCIVIL CODE 465 UNWORTHINESS OF THE PARENT One who is excluded by indignity from the succession does not have on the property thereof which is devolved to his children, the rights of usufruct or administration which the law grants to parents.', metadata={'source': 'inheritance\\INHERITANCE.txt'}),
 Document(page_content='\nCIVIL CODE 466 REHABILITATION OF THE UNWORTHY A person who has incurred indignity is admitted to succession when the person, whose succession is involved, has expressly enabled him to do so by public deed or will. However, the unworthy person who is not expressly authorized, if he was contemplated in the will when the testator knew the cause of the unworthiness, is admitted to succession within the limits of the testamentary provision.', metadata={'source': 'inheritance\\INHERITANCE.txt'}),
 Document(page_content='\nCIVIL CODE 467 NOTION Representation causes descendants to succeed to the place and degree of their ascendant in all cases in wh

In [39]:
div_chunks[10:20]

[Document(page_content="\nCIVIL CODE 170 EXECUTION ON FUND PROPERTY Execution on the fund's property or its fruits can not be levied for debts which the creditor knew to have been contracted for purposes extraneous to the needs of the family.", metadata={'source': 'divorce\\DIVISION OF ASSETS AFTER DIVORCE.txt'}),
 Document(page_content='\nCIVIL CODE 171 TERMINATION OF THE FUND The destination of the fund terminates upon annulment or dissolution or cessation of the civil effects of the marriage. If there are minor children the fund lasts until the last child comes of age. In such a case the court may, at the request of any interested party, lay down rules for the administration of the fund. Having regard to the economic conditions of the parents and the children and any other circumstances, the court may also assign to the children, in enjoyment or ownership, a share in the property of the estate. If there are no children, the provisions on the dissolution of legal community apply.', m

In [41]:
num_total_characters = sum([len(x.page_content) for x in inh_chunks])

print (f"Now you have {len(inh_chunks)} documents that have an average of {num_total_characters / len(inh_chunks):,.0f} characters (smaller pieces)")

Now you have 305 documents that have an average of 440 characters (smaller pieces)


In [42]:
num_total_characters = sum([len(x.page_content) for x in div_chunks])

print (f"Now you have {len(div_chunks)} documents that have an average of {num_total_characters / len(div_chunks):,.0f} characters (smaller pieces)")

Now you have 50 documents that have an average of 561 characters (smaller pieces)


In [40]:
from langchain.embeddings import CohereEmbeddings

embedder = CohereEmbeddings(model='embed-english-v2.0',cohere_api_key=cohere_api_key)


In [41]:
inh_persist_dir = 'inh_db'
div_persist_dir = 'div_db'

In [42]:
inh_docsearch = Chroma.from_documents(documents=inh_chunks, embedding = embedder,persist_directory=inh_persist_dir)
inh_docsearch.persist()
inh_docsearch = None

In [43]:
inh_docsearch = Chroma(persist_directory=inh_persist_dir,embedding_function=embedder)


In [7]:
inh_retriever = inh_docsearch.as_retriever(search_kwargs={"k": 3})


In [44]:
div_docsearch = Chroma.from_documents(documents=div_chunks, embedding = embedder,persist_directory=div_persist_dir)
div_docsearch.persist()
div_docsearch = None

In [8]:
div_docsearch = Chroma(persist_directory=div_persist_dir,embedding_function=embedder)


In [9]:
div_retriever = div_docsearch.as_retriever(search_kwargs={"k": 3})


In [51]:
div_retriever.get_relevant_documents(query ="currency")

[Document(page_content="\nCIVIL PROCEDURE CODE 791 DIVISION PROJECT DRAWN UP BY THE NOTARY The notary draws up single minutes of the transactions carried out. Having drawn up the draft of the shares and lots, if the parties do not agree on it, the notary transmits the minutes to the examining magistrate within five days of their signature. The judge proceeds as in the penultimate paragraph of the preceding article to set the hearing for the parties' appearance and then issue the measures within his jurisdiction pursuant to article 187. Lots may not be extracted except by order of the court made pursuant to the last paragraph of Article 789 or by a final judgment.", metadata={'source': 'divorce\\DIVISION OF ASSETS AFTER DIVORCE.txt'}),
 Document(page_content="\nCIVIL PROCEDURE CODE 791 DIVISION PROJECT DRAWN UP BY THE NOTARY The notary draws up single minutes of the transactions carried out. Having drawn up the draft of the shares and lots, if the parties do not agree on it, the notary 

In [10]:
llm = Cohere(temperature=0.9,cohere_api_key=cohere_api_key)


In [11]:
topic_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Example Input: {input}\nExample Output: {output}",
)

examples = [
    {"input": "How is the weather today?", "output": "OutOfScope"},
    {"input": "I really like eating pizza!", "output": "OutOfScope"},
    {"input": "Are you a cat-person or a dog-person?", "output": "OutOfScope"},
    {"input": "Do you like mountain or sea?", "output": "OutOfScope"},
    {"input": "Can you suggest the most beatiful cities to visit in Italy?", "output": "OutOfScope"},
    {"input": "What is the currency of Japan?", "output": "OutOfScope"},
    {"input": "How old is Cristiano Ronaldo?", "output": "OutOfScope"},
    {"input": "Can you tell me more about this thing?", "output": "OutOfScope"},
    {"input": "What is America best history site?", "output": "OutOfScope"},
    {"input": "Today I am sad , can you cheer me up?", "output": "OutOfScope"},
    {"input": "Can you tell me a joke?", "output": "OutOfScope"},
    {"input": "I have to divorce from my wife and we have to children , can you give me more information?", "output": "divorce"},
    {"input": "Me and my wife decided to divorce and we want to know what happens to our 5 mutually owned shops", "output": "divorce"},
    {"input": "What happens to a family when the parents divorce and they have 2 house a 1 children involved? ", "output": "divorce"},
    {"input": "My grandmother has passed away recently and she inserted me in her final will , what happens now?", "output": "inheritance"},
    {"input": "My husband died yesterday and my son want to know what should we do with his assets ", "output": "inheritance"},
    {"input": "I am a new legal assistant and I want to learn more about what occurs when a father decide to exclude his son from his will , is there a way to oppose to that?", "output": "inheritance"},
    {"input": "There is a little baby whose parents don't want to feed anymore because they are separating, what can we do to help her?", "output": "divorce"},
    {"input": "I am a very old person and I need to write my will , can you tell me more about how to make a will?", "output": "inheritance"},
]


In [12]:

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples, 
    
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    embedder, 
    
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma, 
    
    # This is the number of examples to produce.
    k=4
)

In [13]:
topics = "divorce,inheritance,OutOfScope"

In [14]:
topic_prompt = FewShotPromptTemplate(
    # The object that will help select examples
    example_selector=example_selector,
    
    # Your prompt
    example_prompt=topic_prompt,
    
    # Customizations that will be added to the top and bottom of your prompt
    prefix="""You will be given a question and you have to understand to which topic it belongs. The possible topics are {topics} and you will provide as output the topic the question belongs too.
If you don't know which topics it belongs you should output 'OutOfScope'. You will be given examples to better understand how to find the right topic.""",
    suffix="Input: {question}\nOutput:",
    
    # What inputs your prompt will receive
    input_variables=["question","topics"],
)

In [15]:
print(topic_prompt.format(question="What is America finest history site?",topics=topics))

You will be given a question and you have to understand to which topic it belongs. The possible topics are divorce,inheritance,OutOfScope and you will provide as output the topic the question belongs too.
If you don't know which topics it belongs you should output 'OutOfScope'. You will be given examples to better understand how to find the right topic.

Example Input: What is America best history site?
Example Output: OutOfScope

Example Input: I have to divorce from my wife and we have to children , can you give me more information?
Example Output: divorce

Example Input: I am a new legal assistant and I want to learn more about what occurs when a father decide to exclude his son from his will , is there a way to oppose to that?
Example Output: inheritance

Example Input: Can you tell me more about this thing?
Example Output: OutOfScope

Input: What is America finest history site?
Output:


In [17]:
llm(topic_prompt.format(question="How many laws regulates the fraud?",topics=topics))

' OutOfScope'

In [83]:
template_inh = """ You are playing the role of an expert legal assistant in the Italian Laws regulating Inheritance.
You will be given the most relevant documents related to a question and you have to examine these documents content and try to summarize an answer based on those.
Do not try to force an answer , just say you don't know if you are not sure.

You will be provided with previous messages of the conversation : 
{chat_history}

You will be also provided with some context to base your answer. This is the part you should focus on when drafting your question.
{context}

Question: {question}

Answer: """

prompt_inh = PromptTemplate(template=template_inh, input_variables=["question","context","chat_history"])

inh_kwargs = {"prompt": prompt_inh}

In [84]:
template_div = """ You are playing the role of an expert legal assistant in the Italian Laws regulating Division of Assests after Divorce.
You will be given the most relevant documents related to a question and you have to examine these documents content and try to summarize an answer based on those.
Do not try to force an answer , just say you don't know if you are not sure.

You will be provided with previous messages of the conversation : 
{chat_history}

You will be also provided with some context to base your answer. This is the part you should focus on when drafting your question.
{context}

Question: {question}

Answer: """

prompt_div = PromptTemplate(template=template_div, input_variables=["question","context","chat_history"])

div_kwargs = {"prompt": prompt_div}

In [18]:
from langchain.chains import LLMChain

topic_chain  = LLMChain(llm=llm,prompt=topic_prompt)

In [25]:
myQuestion="Do you have a soul?"

In [26]:
response = topic_chain.run({"question":myQuestion , "topics":topics})

In [27]:
response

' OutOfScope'

In [85]:
from langchain.memory import ConversationBufferWindowMemory

memory_div = ConversationBufferWindowMemory(memory_key='chat_history', return_messages=True,input_key='question',k=2)
memory_inh = ConversationBufferWindowMemory(memory_key='chat_history', return_messages=True,input_key='question')

In [86]:
divorce_question_chain = LLMChain(llm = llm , prompt=prompt_div)
inheritance_question_chain = LLMChain(llm = llm , prompt=prompt_inh)

In [87]:
from langchain.chains import ConversationalRetrievalChain

inheritance = ConversationalRetrievalChain.from_llm(
    llm = llm,
    chain_type='stuff',
    memory = memory_inh,
    retriever=inh_retriever,
    condense_question_llm=llm,
    return_source_documents=True,
    verbose=True)

divorce = ConversationalRetrievalChain.from_llm(
    llm = llm,
    chain_type='stuff',
    memory = memory_div,
    retriever=div_retriever,
    condense_question_llm=llm,
    return_source_documents=True,
    verbose=True)

In [88]:
type(inheritance)

langchain.chains.conversational_retrieval.base.ConversationalRetrievalChain

In [89]:
response = inheritance( {"question":"What happens when my mother exclude me from her will?"})



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.


CIVIL CODE 465 UNWORTHINESS OF THE PARENT One who is excluded by indignity from the succession does not have on the property thereof which is devolved to his children, the rights of usufruct or administration which the law grants to parents.


CIVIL CODE 465 UNWORTHINESS OF THE PARENT One who is excluded by indignity from the succession does not have on the property thereof which is devolved to his children, the rights of usufruct or administration which the law grants to parents.


CIVIL CODE 568 SUCCESSION OF PARENTS A person who dies without leaving children, nor brothers or sisters or their descendants, is succeeded by his father and mother in equal portions, or by the surviving parent.

Question: What happens when my mother exclude me from h

ValueError: One output key expected, got dict_keys(['answer', 'source_documents'])

In [78]:
response

{'question': 'What happens when my mother exclude me from her will?',
 'chat_history': [HumanMessage(content='What happens when my mother exclude me from her will?', additional_kwargs={}, example=False),
  AIMessage(content=' If your mother excluded you from her will, you would not be entitled to any rights of usufruct or administration over her property. ', additional_kwargs={}, example=False)],
 'answer': ' If your mother excluded you from her will, you would not be entitled to any rights of usufruct or administration over her property. ',
 'source_documents': [Document(page_content='\nCIVIL CODE 465 UNWORTHINESS OF THE PARENT One who is excluded by indignity from the succession does not have on the property thereof which is devolved to his children, the rights of usufruct or administration which the law grants to parents.', metadata={'source': 'inheritance\\INHERITANCE.txt'}),
  Document(page_content='\nCIVIL CODE 465 UNWORTHINESS OF THE PARENT One who is excluded by indignity from 

In [81]:
response = inheritance( {"question":"What happens when my mother exclude me from her will?"})

ValueError: Missing some input keys: {'context'}

In [80]:
response

{'question': 'What happens when my mother exclude me from her will?',
 'chat_history': [HumanMessage(content='What happens when my mother exclude me from her will?', additional_kwargs={}, example=False),
  AIMessage(content=' If your mother excluded you from her will, you would not be entitled to any rights of usufruct or administration over her property. ', additional_kwargs={}, example=False)],
 'answer': ' If your mother excluded you from her will, you would not be entitled to any rights of usufruct or administration over her property. ',
 'source_documents': [Document(page_content='\nCIVIL CODE 465 UNWORTHINESS OF THE PARENT One who is excluded by indignity from the succession does not have on the property thereof which is devolved to his children, the rights of usufruct or administration which the law grants to parents.', metadata={'source': 'inheritance\\INHERITANCE.txt'}),
  Document(page_content='\nCIVIL CODE 465 UNWORTHINESS OF THE PARENT One who is excluded by indignity from 

In [26]:
def extract_source_documents(response : dict):
    source_documents = [elem.page_content for elem in response["source_documents"]]
    relevant_documents =[" ".join(elem.split()[:3]) for elem in source_documents]
    return relevant_documents

def extract_text(response):
    return response["text"]


In [39]:
relevant_documents = extract_source_documents(response)
relevant_documents

['CIVIL CODE 696', 'CIVIL CODE 734', 'CIVIL CODE 583']

In [27]:
divorce = RetrievalQA.from_chain_type(llm = llm ,chain_type="stuff",retriever=div_retriever,chain_type_kwargs=div_kwargs)

In [29]:
out_examples = [
  {
    "question": "How is the weather today?",
    "answer": 
"""
I am a virtual assistant specialized in the Italian Civil Code regulating Inheritance or Division of Assests after Divorce , 
ask a question in the context of these two topics. If your question was related to such topics , please try to rephrase it.
"""
  },
  {
    "question": "Can you suggest the most beatiful cities to visit in Italy?",
    "answer": 
"""
I am a virtual assistant specialized in the Italian Civil Code regulating Inheritance or Division of Assests after Divorce , 
ask a question in the context of these two topics. If your question was related to such topics , please try to rephrase it.
"""
  },
  {
    "question": "What is the currency of Japan?",
    "answer":
"""
I am a virtual assistant specialized in the Italian Civil Code regulating Inheritance or Division of Assests after Divorce , 
ask a question in the context of these two topics. If your question was related to such topics , please try to rephrase it.
"""
  }
]

In [30]:
out_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Example Input: {question}\nExample Output: {answer}",
)

In [31]:
example_selector_out = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    out_examples, 
    
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    embedder, 
    
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma, 
    
    # This is the number of examples to produce.
    k=3
)

In [32]:
out_few_prompt = FewShotPromptTemplate(
    example_selector=example_selector_out,
    example_prompt=out_prompt, 
    suffix="Question: {question}", 
    input_variables=["question"]
)

In [33]:
outofscope = LLMChain(llm=llm,prompt=out_few_prompt)

In [70]:
print(out_few_prompt.format(question="What is the band name whose singer is called Robert Plant?"))

Example Input: How is the weather today?
Example Output: 
I am a virtual assistant specialized in the Italian Civil Code regulating Inheritance or Division of Assests after Divorce , 
ask a question in the context of these two topics. If your question was related to such topics , please try to rephrase it.


Example Input: Can you suggest the most beatiful cities to visit in Italy?
Example Output: 
I am a virtual assistant specialized in the Italian Civil Code regulating Inheritance or Division of Assests after Divorce , 
ask a question in the context of these two topics. If your question was related to such topics , please try to rephrase it.


Example Input: What is the currency of Japan?
Example Output: 
I am a virtual assistant specialized in the Italian Civil Code regulating Inheritance or Division of Assests after Divorce , 
ask a question in the context of these two topics. If your question was related to such topics , please try to rephrase it.


Question: What is the band name

In [73]:
response = outofscope({"question":"What is the color of the sky?"})

In [74]:
print(response)

{'question': 'What is the color of the sky?', 'text': '\nI am a virtual assistant specialized in the Italian Civil Code regulating Inheritance or Division of Assests after Divorce , \nask a question in the context of these two topics. If your question was related to such topics , please try to rephrase it.'}


Now we have our chains , let's define the agent

In [36]:
def run_chain(input,topics):
    topic = topic_chain({"question":input,"topics":topics})
    if "divorce" in topic["text"]:
        response =  divorce({"query":input})
    elif "inheritance" in topic["text"]:
        response = inheritance({"query":input})
    else:
        response = outofscope({"question":input})

    return response

In [58]:
def get_topic(input,topics):
    return topic_chain({"question":input,"topics":topics})

In [37]:
response = run_chain("What happens when I die without making a will?",topics)

Your text contains a trailing whitespace, which has been trimmed to ensure high quality generations.


In [38]:
response

{'query': 'What happens when I die without making a will?',
 'result': '\nIn the case of dying without a will, all assets will be given to the closest relatives with legal rights.',
 'source_documents': [Document(page_content='\nCIVIL CODE 696 DEVOLUTION TO THE SUBSTITUTED The estate shall devolve to the substitute upon the death of the instituted person. If the persons or entities that took care of the incapacitated person die or become extinct before his or her death, the property or the portion of the property that would accrue to them shall devolve to the legal successors of the incapacitated person.', metadata={'source': 'inheritance\\INHERITANCE.txt'}),
  Document(page_content="\nCIVIL CODE 734 DIVISION MADE BY THE TESTATOR The testator might divide his property among the heirs by including in the division also the unavailable part. If the division made by the testator does not include all the property left at the time of death, the property not included therein shall be attribut

In [65]:
topic = get_topic("What happens when I die without making a will?",topics)

In [66]:
topic


{'question': 'What happens when I die without making a will?',
 'topics': 'divorce,inheritance,OutOfScope',
 'text': ' inheritance'}

In [68]:
"inheritance" in topic

False

In [ ]:
from langchain.agents import Tool
tools = [
    Tool(
        name='Inheritance search',
        func=inheritance.run,
        description='Useful when you need to answer questions about the Italian laws regulating the process of inheritance. Always give the source article names and number your question is based on. The input should be a question written given by a non-expert legal person regarding how inheritance works in particular cases'
    ),
    Tool(
        name='Divorce Search',
        func=divorce.run,
        description='Useful when you need to answer questions about the Italian laws regulation the division of assets after divorce. Always give the source article names and number your question is based on. The input should be a question written given by a non-expert legal person regarding how division of assets after divorce works in particular cases'
    ),
    Tool(
        name='Out-of-scope answer',
        func=outofscope.run,
        description = 'Useful when the question asked it is not related neither to division of assets after divorce or inheritance.Always remind to the user that he is going out of scope.'
    )


]

In [ ]:
tool_names=[]
for tool in tools:
    tool_names.append(tool.name)

In [ ]:
tool_names

In [ ]:
PREFIX = """You are an expert legal assistant about the Italian Laws regulating inheritance and division of assets."""
FORMAT_INSTRUCTIONS = """Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question. Always quote the articles your question is based on."""
SUFFIX = """Begin!

Question: {input}
Thought:{agent_scratchpad}"""

In [ ]:
from langchain.agents import initialize_agent
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True, return_intermediate_steps=True,agent_kwargs={
        'prefix':PREFIX,
        'format_instructions':FORMAT_INSTRUCTIONS,
        'suffix':SUFFIX,
    })

In [ ]:
response = agent({"input":"What happens when I divorce my wife?",
                    "tool_names":tool_names})

# STREAMLIT UI # 

In [61]:
import streamlit as st
from streamlit_chat import message

topics = "divorce,inheritance,OutOfScope"

def run_chain(input,topics):
    topic = topic_chain({"question":input,"topics":topics})
    if "divorce" in topic:
        response =  divorce({"question":input})
    elif "inheritance" in topic:
        response = inheritance({"question":input})
    else:
        response = outofscope({"question":input})

    return response

def get_topic(input,topics):
    return topic_chain({"question":input,"topics":topics})

def extract_source_documents(response : dict):
    source_documents = [elem.page_content for elem in response["source_documents"]]
    relevant_documents =[" ".join(elem.split()[:3]) for elem in source_documents]
    return relevant_documents

def extract_text(response):
    return response["text"]


st.set_page_config(page_title="LangChain Demo", page_icon=":robot:")
st.header("LangChain Demo")

if "generated" not in st.session_state:
    st.session_state["generated"] = []

if "past" not in st.session_state:
    st.session_state["past"] = []


def get_text():
    input_text = st.text_input("You: ", "Hello, how are you?", key="input")
    return input_text


user_input = get_text()

if user_input:
    output = run_chain(user_input,topics)

    st.session_state.past.append(user_input)
    st.session_state.generated.append(output)

if st.session_state["generated"]:

    for i in range(len(st.session_state["generated"]) - 1, -1, -1):
        message(st.session_state["generated"][i], key=str(i))
        message(st.session_state["past"][i], is_user=True, key=str(i) + "_user")

2023-07-17 13:34:58.099 
  command:

    streamlit run c:\PROGETTI\uni\myChatBot\chat\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2023-07-17 13:34:58.100 Session state does not function when running a script without `streamlit run`


AttributeError: st.session_state has no attribute "past". Did you forget to initialize it? More info: https://docs.streamlit.io/library/advanced-features/session-state#initialization